In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector

In [2]:
def movie(idx, img_save = False):
    url = f'http://www.cine21.com/movie/info/?movie_id={idx}'
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        if '존재하지 않는 영화입니다.' in soup.find('script').text:
            # print(f'(생략) 인덱스 {idx} : 존재하지 않는 영화입니다')
            return f'{idx} 존재하지 않는 영화'
        elif '성인용 컨텐츠입니다.' in soup.find('script').text:
            # print(f'(생략) 인덱스 {idx} : 성인용 영화입니다')
            return f'{idx} 성인용 영화'
        movie_info = soup.find('div', class_ = 'mov_info')
        span = movie_info.find_all('span')
    except:
        # print(f'(생략) 인덱스 {idx} : 서버 장애 발생')
        return f'{idx} 서버 장애'

    try:
        movie_name_ko = movie_info.find('p', class_ = 'tit').text
        movie_name_en = span[-8].text
        country = span[-6].text
        grade = span[-5].text
        genre = span[-4].text
        time = int(span[-3].text.split()[-1][:-1])
        date = span[-2].text.split()[-1]
        audience = int(span[-1].text.split()[-1][:-1].replace(',',''))
        director = movie_info.find('a').text
        poster_link = soup.find('div', class_ = 'mov_poster').find('img')['src']

        if country.isnumeric() == True:
            return f'{idx} 영화 정보 불충분'

        if poster_link == 'https://image.cine21.com/resize//noimg[X230,330].jpg':
            # print(f"(생략) 인덱스 {idx} : 포스터 정보 불충분")
            return f'{idx} 포스터 없음'

        if img_save == True:
            poster_response = requests.get(poster_link)
            poster_image = Image.open(BytesIO(poster_response.content))
            poster_image.save(f'/Users/anhyojun/WorkSpace/LocalData/movie_poster/{idx}_{movie_name_ko}.jpg')
        # print(movie_name_ko, movie_name_en, country, grade, genre, time, date, audience, director, poster_link)

        return idx, movie_name_ko, movie_name_en, country, grade, date, time, audience, director, poster_link, genre

    except:
        # print(f"(생략) 인덱스 {idx} : 영화 정보 불충분")
        return f'{idx} 영화 정보 불충분'

In [3]:
# My SQL 데이터베이스 연결
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'signcity114',
    database = 'MyDB'
)

# 연결 확인
if conn.is_connected():
    print("MySQL에 성공적으로 연결되었습니다")
else:
    print("MySQL 연결 실패")

# 연결 종료
# conn.close()

MySQL에 성공적으로 연결되었습니다


In [4]:
create_table_query = """
CREATE TABLE IF NOT EXISTS movies (
    id INT PRIMARY KEY,
    movie_name_ko VARCHAR(255),
    movie_name_en VARCHAR(255),
    country VARCHAR(255),
    grade VARCHAR(255),
    release_date DATE,
    runtime INT,
    audience INT,
    director VARCHAR(255),
    poster_link VARCHAR(255),
    genre VARCHAR(255))"""

insert_query = """
INSERT INTO movies
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [5]:
# cursor = conn.cursor()

# cursor.execute(create_table_query)
# conn.commit()
# print("테이블이 성공적으로 생성되었습니다")

# conn.close()

In [ ]:
cursor = conn.cursor()

data = movie(1)

In [ ]:
for i in range(1, 101):
    cursor = conn.cursor()

    data = movie(i)
    if len(data) == 11:
        cursor.execute(insert_query, data)
        print(data)
conn.commit()

(1, '히트', 'Heat', '미국', '청소년 관람불가', '2017-11-09', 171, 412819, '마이클 만', 'https://image.cine21.com/resize/cine21/poster/2017/1024/13_51_32__59eec6d4af7bb[X230,330].jpg', '액션, 범죄, 드라마')
